<a href="https://colab.research.google.com/github/Alan-Cheong/IEEE_QW_2020/blob/master/Crypto_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
class Cryptocurrency:
    def __init__(self, max_supply, initial_reward, halving_interval):
        """
        Initialize the cryptocurrency parameters.
        :param max_supply: The maximum number of coins allowed.
        :param initial_reward: The initial reward per block mined.
        :param halving_interval: The number of blocks after which the reward is halved.
        """
        self.max_supply = max_supply
        self.current_supply = 0
        self.reward = initial_reward
        self.halving_interval = halving_interval
        self.blocks_mined = 0

    def mine_block(self):
        """
        Simulate mining a block and adjusting the reward if necessary.
        :return: The reward for mining the block, or 0 if max supply is reached.
        """
        if self.current_supply >= self.max_supply:
            print("Max supply reached. No more coins can be mined.")
            return 0

        # Add the current reward to the total supply
        coins_mined = min(self.reward, self.max_supply - self.current_supply)
        self.current_supply += coins_mined
        self.blocks_mined += 1

        # Adjust reward if halving interval is reached
        if self.blocks_mined % self.halving_interval == 0:
            self.reward /= 2

        return coins_mined

    def get_status(self):
        """
        Get the current status of the cryptocurrency.
        :return: A dictionary containing current stats.
        """
        return {
            "Current Supply": self.current_supply,
            "Reward per Block": self.reward,
            "Blocks Mined": self.blocks_mined,
            "Remaining Supply": self.max_supply - self.current_supply,
        }


# Example usage
if __name__ == "__main__":
    # Define cryptocurrency parameters
    MAX_SUPPLY = 1000  # Maximum coin supply
    INITIAL_REWARD = 50      # Initial block reward
    HALVING_INTERVAL = 2  # Blocks per halving (Bitcoin uses this value)

    # Create cryptocurrency
    crypto = Cryptocurrency(MAX_SUPPLY, INITIAL_REWARD, HALVING_INTERVAL)

    # Simulate mining
    for _ in range(10):  # Simulate 10 blocks being mined
        reward = crypto.mine_block()
        print(f"Block {_ + 1} mined! Reward: {reward} coins")

    # Display final status
    print("\nFinal Cryptocurrency Status:")
    print(crypto.get_status())


Block 1 mined! Reward: 50 coins
Block 2 mined! Reward: 50 coins
Block 3 mined! Reward: 25.0 coins
Block 4 mined! Reward: 25.0 coins
Block 5 mined! Reward: 12.5 coins
Block 6 mined! Reward: 12.5 coins
Block 7 mined! Reward: 6.25 coins
Block 8 mined! Reward: 6.25 coins
Block 9 mined! Reward: 3.125 coins
Block 10 mined! Reward: 3.125 coins

Final Cryptocurrency Status:
{'Current Supply': 193.75, 'Reward per Block': 1.5625, 'Blocks Mined': 10, 'Remaining Supply': 806.25}


In [4]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

import hashlib

class CryptocurrencyWithECDSA:
    def __init__(self, max_supply, initial_reward, halving_interval):
        """
        Initialize the cryptocurrency with ECDSA.
        :param max_supply: Maximum number of coins.
        :param initial_reward: Initial reward per mined block.
        :param halving_interval: Number of blocks after which reward halves.
        """
        self.max_supply = max_supply
        self.current_supply = 0
        self.reward = initial_reward
        self.halving_interval = halving_interval
        self.blocks_mined = 0

        # Generate an ECDSA private key and the corresponding public key
        self.private_key = ec.generate_private_key(ec.SECP256K1(), default_backend())
        self.public_key = self.private_key.public_key()

    def mine_block(self, block_data):
        """
        Simulate mining a block, signing it with the private key, and verifying with the public key.
        :param block_data: Data to be included in the block.
        :return: The reward for mining or 0 if max supply is reached.
        """
        if self.current_supply >= self.max_supply:
            print("Max supply reached. No more coins can be mined.")
            return 0

        # Sign the block data with the private key
        signature = self.sign_block(block_data)

        # Verify the signature with the public key
        if not self.verify_block_signature(block_data, signature):
            print("Block signature verification failed.")
            return 0

        # Add coins to supply
        coins_mined = min(self.reward, self.max_supply - self.current_supply)
        self.current_supply += coins_mined
        self.blocks_mined += 1

        # Halve the reward if necessary
        if self.blocks_mined % self.halving_interval == 0:
            self.reward /= 2

        return coins_mined

    def sign_block(self, block_data):
        """
        Sign the block data with the private key.
        :param block_data: The data to sign.
        :return: The signature for the block.
        """
        signature = self.private_key.sign(
            block_data.encode(),
            ec.ECDSA(hashes.SHA256())
        )
        return signature

    def verify_block_signature(self, block_data, signature):
        """
        Verify the block data's signature using the public key.
        :param block_data: The block data to verify.
        :param signature: The signature of the block data.
        :return: True if valid, False otherwise.
        """
        try:
            self.public_key.verify(
                signature,
                block_data.encode(),
                ec.ECDSA(hashes.SHA256())
            )
            return True
        except Exception as e:
            print(f"Verification failed: {e}")
            return False

    def get_status(self):
        """
        Get the current status of the cryptocurrency.
        :return: A dictionary containing the current status.
        """
        return {
            "Current Supply": self.current_supply,
            "Reward per Block": self.reward,
            "Blocks Mined": self.blocks_mined,
            "Remaining Supply": self.max_supply - self.current_supply,
        }


# Example usage
if __name__ == "__main__":
    # Define cryptocurrency parameters
    MAX_SUPPLY = 1000  # Maximum coin supply (Originally set at 21 mil for Bitcoin)
    INITIAL_REWARD = 50      # Initial block reward
    HALVING_INTERVAL = 2  # Blocks per halving (Bitcoin uses 210,000 value)

    # Create cryptocurrency with ECDSA
    crypto = CryptocurrencyWithECDSA(MAX_SUPPLY, INITIAL_REWARD, HALVING_INTERVAL)

    # Simulate mining some blocks
    block_data = "Block 1 Data"
    for _ in range(10):
        reward = crypto.mine_block(block_data)
        print(f"Block {_ + 1} mined! Reward: {reward} coins")

    # Display final status
    print("\nFinal Cryptocurrency Status:")
    print(crypto.get_status())


Block 1 mined! Reward: 50 coins
Block 2 mined! Reward: 50 coins
Block 3 mined! Reward: 25.0 coins
Block 4 mined! Reward: 25.0 coins
Block 5 mined! Reward: 12.5 coins
Block 6 mined! Reward: 12.5 coins
Block 7 mined! Reward: 6.25 coins
Block 8 mined! Reward: 6.25 coins
Block 9 mined! Reward: 3.125 coins
Block 10 mined! Reward: 3.125 coins

Final Cryptocurrency Status:
{'Current Supply': 193.75, 'Reward per Block': 1.5625, 'Blocks Mined': 10, 'Remaining Supply': 806.25}


In [6]:
# Pseudocode for Lattice Cryptography Integration (Conceptual)
from pyntru import PyNTRU # Hypothetical library

class LatticeCryptocurrency:
    def __init__(self, max_supply, initial_reward, halving_interval):
        """
        Initialize cryptocurrency with NTRU-based encryption.
        :param max_supply: Maximum number of coins.
        :param initial_reward: Initial reward per mined block.
        :param halving_interval: Number of blocks after which reward halves.
        """
        self.max_supply = max_supply
        self.current_supply = 0
        self.reward = initial_reward
        self.halving_interval = halving_interval
        self.blocks_mined = 0

        # Generate NTRU private and public key pair
        self.private_key, self.public_key = NTRU.generate_keys()

    def mine_block(self, block_data):
        """
        Simulate mining a block, encrypting the block data with lattice encryption,
        and verifying it with the public key.
        :param block_data: Data to be included in the block.
        :return: The reward for mining or 0 if max supply is reached.
        """
        if self.current_supply >= self.max_supply:
            print("Max supply reached. No more coins can be mined.")
            return 0

        # Encrypt the block data with NTRU encryption (hypothetical encryption step)
        encrypted_block_data = self.encrypt_block_data(block_data)

        # Decrypt the block data using NTRU public key (hypothetical decryption step)
        decrypted_data = self.decrypt_block_data(encrypted_block_data)

        if decrypted_data != block_data:
            print("Block data integrity verification failed.")
            return 0

        # Add coins to supply
        coins_mined = min(self.reward, self.max_supply - self.current_supply)
        self.current_supply += coins_mined
        self.blocks_mined += 1

        # Halve the reward if necessary
        if self.blocks_mined % self.halving_interval == 0:
            self.reward /= 2

        return coins_mined

    def encrypt_block_data(self, block_data):
        """
        Encrypt block data with NTRU encryption.
        :param block_data: Data to encrypt.
        :return: Encrypted data.
        """
        encrypted_data = NTRU.encrypt(block_data, self.public_key)
        return encrypted_data

    def decrypt_block_data(self, encrypted_data):
        """
        Decrypt block data using NTRU private key.
        :param encrypted_data: Encrypted data to decrypt.
        :return: Decrypted block data.
        """
        decrypted_data = NTRU.decrypt(encrypted_data, self.private_key)
        return decrypted_data

    def get_status(self):
        """
        Get the current status of the cryptocurrency.
        :return: A dictionary containing the current status.
        """
        return {
            "Current Supply": self.current_supply,
            "Reward per Block": self.reward,
            "Blocks Mined": self.blocks_mined,
            "Remaining Supply": self.max_supply - self.current_supply,
        }


ModuleNotFoundError: No module named 'pyntru'